In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Pip install requirements

In [ ]:
!pip install --user flair

In [ ]:
!pip install vaderSentiment

# Read csv

In [ ]:
df = pd.read_csv('../input/retinol/cream2k.csv')
df=df.dropna()
df.head(20)

# Vader

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
def sentiment_scores(sentence,i,v_list,dict_list):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    if sentiment_dict['compound'] >= 0.1 :
        v_list.append(1)
        dict_list.append(sentiment_dict['compound'])
    elif sentiment_dict['compound'] < 0.1:
        v_list.append(0)
        dict_list.append(sentiment_dict['compound'])

In [ ]:
i=0
v_list=[]
dict_list=[]
for sentence in df['Review']:
    sentiment_scores(sentence,i,v_list,dict_list)
    i=i+1

In [ ]:
df['Vader']=v_list
df['sentiment_dict_sompound']=dict_list

**Obtaining Accuracy**

In [ ]:
s=0
i=0
for manual in df["Sug/Com"]:
    if(manual==df["Vader"][i]):
        s=s+1
    i=i+1
accuracy=(s/i)*100
print("Accuracy Vader:",accuracy) 


# TEXTBLOB

In [ ]:
from textblob import TextBlob
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
def subjective(text):
        return TextBlob(text).sentiment.subjectivity
        
def polarity(text):
        return TextBlob(text).sentiment.polarity
        
def sentiment_analysis(review):
    review['TextBlob_Subjectivity'] = review['Review'].apply(subjective)
    review['TextBlob_Polarity'] = review['Review'].apply(polarity)
    return review
df4=sentiment_analysis(df)
j=0
for i in df4['TextBlob_Polarity']:
    if i<0.1:
        df4['TextBlob_Polarity'][j]=0
        j=j+1
    else:
        df4['TextBlob_Polarity'][j]=1
        j=j+1

**Obtaining Accuracy**

In [ ]:
s=0
i=0
for manual in df4["Sug/Com"]:
    if(manual==df4["TextBlob_Polarity"][i]):
        s=s+1
    i=i+1
accuracy=(s/i)*100
print("Accuracy is",accuracy)


# FLAIR

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence
sia = TextClassifier.load('en-sentiment')
def flair_prediction(x):
    sentence = Sentence(x)
    sia.predict(sentence)
    score = sentence.labels[0]
    if "POSITIVE" in str(score):
        return 1
    elif "NEGATIVE" in str(score):
        return 0
    else:
        return 1
df["flair_sentiment"] = df["Review"].apply(flair_prediction)

**Obtaining Accuracy**

In [ ]:
s=0
i=0
for manual in df["Sug/Com"]:
    if(manual==df["flair_sentiment"][i]):
        s=s+1
    i=i+1
accuracy=(s/i)*100
print("Accuracy Flair:",accuracy) 


In [ ]:
df.to_csv("retinol.csv")